In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import kagglehub
import os
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
import numpy as np
import pandas as pd
from tqdm import tqdm

nikhilroxtomar_person_segmentation_path = kagglehub.dataset_download('nikhilroxtomar/person-segmentation')
tapakah68_supervisely_filtered_segmentation_person_dataset_path = kagglehub.dataset_download('tapakah68/supervisely-filtered-segmentation-person-dataset')

print('Data source import complete.')
print(nikhilroxtomar_person_segmentation_path)
print(tapakah68_supervisely_filtered_segmentation_person_dataset_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using Colab cache for faster access to the 'person-segmentation' dataset.
Using Colab cache for faster access to the 'supervisely-filtered-segmentation-person-dataset' dataset.
Data source import complete.
/kaggle/input/person-segmentation
/kaggle/input/supervisely-filtered-segmentation-person-dataset


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:1]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/person-segmentation/people_segmentation/README
/kaggle/input/person-segmentation/people_segmentation/segmentation/val.txt
/kaggle/input/person-segmentation/people_segmentation/images/pexels-photo-219004.jpg
/kaggle/input/person-segmentation/people_segmentation/masks/girl-beautiful-beautiful-girl-face.png
/kaggle/input/supervisely-filtered-segmentation-person-dataset/df.csv
/kaggle/input/supervisely-filtered-segmentation-person-dataset/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img/collage/ds6_pexels-photo-792385.jpg
/kaggle/input/supervisely-filtered-segmentation-person-dataset/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img/images/ds8_pexels-photo-838817.png
/kaggle/input/supervisely-filtered-segmentation-person-dataset/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img/masks/ds8_pexels-photo-838817.png


In [ ]:
images_path =  "/kaggle/input/supervisely-filtered-segmentation-person-dataset/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img/"
masks_path  = "/kaggle/input/supervisely-filtered-segmentation-person-dataset/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img/"

images_path2="/kaggle/input/person-segmentation/people_segmentation/images/"
masks_path2="/kaggle/input/person-segmentation/people_segmentation/masks/"
df =  pd.read_csv('/kaggle/input/supervisely-filtered-segmentation-person-dataset/df.csv')
df.head()
images_path2_list = sorted(os.listdir(images_path2))
masks_path2_list = sorted(os.listdir(masks_path2))
df2 = df[["images", "masks"]].copy()
df2["images"] = df2['images'].apply(lambda x: images_path + x)
df2["masks"]  = df2['masks'].apply(lambda x: masks_path + x)
df2["coef"]   = 1

df3 = pd.DataFrame({
    "images": [images_path2 + elt for elt in images_path2_list],
    "masks":  [masks_path2 + elt for elt in masks_path2_list],
    "coef":   255
})

final_df = pd.concat([df2, df3], ignore_index=True)

X_train_raw, X_test_raw  =  train_test_split(final_df, test_size=0.1, random_state=42)

In [ ]:
final_df = final_df.copy()
final_df["source"] = np.where(final_df["coef"] == 1, "supervisely", "person_seg")
print("Tổng số mẫu:", len(final_df))
print("\nSố mẫu theo nguồn:")
print(final_df["source"].value_counts())

Tổng số mẫu: 8345

Số mẫu theo nguồn:
source
person_seg     5678
supervisely    2667
Name: count, dtype: int64


In [ ]:
def dice_loss(logits, target, eps=1e-6):
    pred = torch.sigmoid(logits)
    target = target.float()
    intersection = (pred * target).sum()
    return 1 - (2. * intersection + eps) / (pred.sum() + target.sum() + eps)

def dice_score_from_logits(logits, target, thr=0.5, eps=1e-6):
    prob = torch.sigmoid(logits)
    pred = (prob > thr).float()
    inter = (pred * target).sum()
    union = pred.sum() + target.sum()
    return ((2*inter + eps) / (union + eps)).item()

def logits_to_probs_preds(logits, thr=0.5):
    probs = torch.sigmoid(logits)
    preds = (probs > thr).float()
    return probs, preds

def batch_pixel_accuracy(preds, targets):
    correct = (preds == targets).float().sum()
    total = torch.numel(targets)
    return (correct / total).item()

def batch_iou(preds, targets, eps=1e-6):
    inter = (preds * targets).sum(dim=(1,2,3))
    union = (preds + targets - preds*targets).sum(dim=(1,2,3))
    iou_per_image = ((inter + eps) / (union + eps))
    return iou_per_image.mean().item()

def batch_dice(preds, targets, eps=1e-6):
    inter = (preds * targets).sum(dim=(1,2,3))
    denom = preds.sum(dim=(1,2,3)) + targets.sum(dim=(1,2,3))
    dice_per_image = ((2*inter + eps) / (denom + eps))
    return dice_per_image.mean().item()

def batch_precision_recall_f1(preds, targets, eps=1e-6):
    tp = (preds * targets).sum(dim=(1,2,3))
    fp = (preds * (1 - targets)).sum(dim=(1,2,3))
    fn = ((1 - preds) * targets).sum(dim=(1,2,3))

    precision = (tp + eps) / (tp + fp + eps)
    recall = (tp + eps) / (tp + fn + eps)
    f1 = (2 * precision * recall + eps) / (precision + recall + eps)

    return precision.mean().item(), recall.mean().item(), f1.mean().item()


In [ ]:
#RAW TRAIN
class RawSegDataset(Dataset):
    def __init__(self, df, size=512):
        self.df = df.reset_index(drop=True)
        self.size = size

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_path  = row["images"]
        mask_path = row["masks"]
        coef      = row.get("coef", 1)

        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        img = img.resize((self.size, self.size), Image.BILINEAR)
        mask = mask.resize((self.size, self.size), Image.NEAREST)

        # --- Convert to numpy ---
        img_np = np.array(img, dtype=np.float32)        # 0..255
        mask_np = np.array(mask, dtype=np.float32)      # 0..255 (likely)

        # --- Robust normalization for mask ---
        # Normalize mask to [0,1] regardless of original encoding:
        max_val = mask_np.max() if mask_np.max() > 0 else 1.0
        if max_val > 1.0:
            mask_np = mask_np / max_val   # now in [0,1]
        mask_bin = (mask_np >= 0.5).astype(np.float32)

        # --- Image scaling + ImageNet normalization ---
        img_np = img_np / 255.0   # to [0,1]

        # to tensor (C,H,W)
        img_t = torch.from_numpy(img_np).permute(2,0,1).float()
        mask_t = torch.from_numpy(mask_bin).unsqueeze(0).float()

        # ImageNet normalize
        mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32).view(3,1,1)
        std  = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32).view(3,1,1)
        img_t = (img_t - mean) / std

        return img_t, mask_t

class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 1)
        self.conv6 = nn.Conv2d(in_channels, out_channels, 3, padding=6, dilation=6)
        self.conv12 = nn.Conv2d(in_channels, out_channels, 3, padding=12, dilation=12)
        self.conv18 = nn.Conv2d(in_channels, out_channels, 3, padding=18, dilation=18)

        self.global_pool = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, out_channels, 1),
            nn.ReLU(inplace=True)
        )

        self.out_conv = nn.Conv2d(out_channels * 5, out_channels, 1)

    def forward(self, x):
        h, w = x.shape[2], x.shape[3]
        y1 = self.conv1(x)
        y2 = self.conv6(x)
        y3 = self.conv12(x)
        y4 = self.conv18(x)
        y5 = self.global_pool(x)
        y5 = F.interpolate(y5, size=(h, w), mode="bilinear", align_corners=False)
        y = torch.cat([y1, y2, y3, y4, y5], dim=1)
        return self.out_conv(y)

class DeepLabV3_MobileNetV3Large(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        base = mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT)
        self.backbone = base.features
        backbone_out = 960
        self.aspp = ASPP(backbone_out, 256)
        self.decoder = nn.Sequential(
            nn.Conv2d(256, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, num_classes, 1)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.aspp(x)
        x = self.decoder(x)
        x = F.interpolate(x, size=(512, 512), mode="bilinear", align_corners=False)
        return x

train_ds = RawSegDataset(X_train_raw, size=512)
val_ds   = RawSegDataset(X_test_raw,  size=512)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=8, shuffle=False, num_workers=2, pin_memory=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepLabV3_MobileNetV3Large(num_classes=1).to(device)

criterion_bce = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20
history = {
    "train_loss": [],
    "val_loss": [],
    "val_iou": [],
    "val_dice": [],
    "val_acc": [],
    "val_prec": [],
    "val_recall": [],
    "val_f1": []
}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_iou = 0.0
    train_dice = 0.0
    train_acc = 0.0
    train_steps = 0

    for imgs, masks in tqdm(train_loader, desc=f"Train epoch {epoch+1}"):
        imgs = imgs.to(device).float()
        masks = masks.to(device).float()

        logits = model(imgs)
        bce = criterion_bce(logits, masks)
        dsc = dice_loss(logits, masks)
        loss = bce + dsc

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        with torch.no_grad():
            _, preds = logits_to_probs_preds(logits, thr=0.5)
            train_iou += batch_iou(preds, masks)
            train_dice += batch_dice(preds, masks)
            train_acc += batch_pixel_accuracy(preds, masks)
            train_steps += 1

    avg_loss = running_loss / max(1, len(train_loader))
    avg_train_iou = train_iou / max(1, train_steps)
    avg_train_dice = train_dice / max(1, train_steps)
    avg_train_acc = train_acc / max(1, train_steps)
    print(f"Epoch {epoch+1} - train avg loss: {avg_loss:.4f} | IoU: {avg_train_iou:.4f} | Dice: {avg_train_dice:.4f} | Acc: {avg_train_acc:.4f}")
    history["train_loss"].append(avg_loss)

    model.eval()
    val_loss = 0.0
    val_steps = 0

    sum_iou = 0.0
    sum_dice = 0.0
    sum_acc = 0.0
    sum_prec = 0.0
    sum_recall = 0.0
    sum_f1 = 0.0

    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs = imgs.to(device).float()
            masks = masks.to(device).float()

            logits = model(imgs)
            bce = criterion_bce(logits, masks)
            dsc = dice_loss(logits, masks)
            batch_loss = (bce + dsc).item()
            val_loss += batch_loss

            probs, preds = logits_to_probs_preds(logits, thr=0.5)

            # metrics
            iou_b = batch_iou(preds, masks)
            dice_b = batch_dice(preds, masks)
            acc_b = batch_pixel_accuracy(preds, masks)
            prec_b, recall_b, f1_b = batch_precision_recall_f1(preds, masks)

            sum_iou += iou_b
            sum_dice += dice_b
            sum_acc += acc_b
            sum_prec += prec_b
            sum_recall += recall_b
            sum_f1 += f1_b

            val_steps += 1

    avg_val_loss = val_loss / max(1, val_steps)
    avg_val_iou = sum_iou / max(1, val_steps)
    avg_val_dice = sum_dice / max(1, val_steps)
    avg_val_acc = sum_acc / max(1, val_steps)
    avg_val_prec = sum_prec / max(1, val_steps)
    avg_val_recall = sum_recall / max(1, val_steps)
    avg_val_f1 = sum_f1 / max(1, val_steps)

    history["val_loss"].append(avg_val_loss)
    history["val_iou"].append(avg_val_iou)
    history["val_dice"].append(avg_val_dice)
    history["val_acc"].append(avg_val_acc)
    history["val_prec"].append(avg_val_prec)
    history["val_recall"].append(avg_val_recall)
    history["val_f1"].append(avg_val_f1)

    print(f"Val loss: {avg_val_loss:.4f} | IoU: {avg_val_iou:.4f} | Dice: {avg_val_dice:.4f} | Acc: {avg_val_acc:.4f}")
    print(f"Precision: {avg_val_prec:.4f} | Recall: {avg_val_recall:.4f} | F1: {avg_val_f1:.4f}")

import pandas as pd
hist_df = pd.DataFrame(history)
hist_df.to_csv("/content/drive/MyDrive/Data Mining/Project/Model/training_history_raw_metrics.csv", index=False)
print("Saved training history to Drive (training_history_raw_metrics.csv)")

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth


100%|██████████| 21.1M/21.1M [00:00<00:00, 182MB/s]
Train epoch 1: 100%|██████████| 939/939 [05:18<00:00,  2.95it/s]

Epoch 1 - train avg loss: 0.2607 | IoU: 0.7841 | Dice: 0.8644 | Acc: 0.9484


Val loss: 0.1726 | IoU: 0.8361 | Dice: 0.9022 | Acc: 0.9669
Precision: 0.9166 | Recall: 0.8982 | F1: 0.9022


Train epoch 2: 100%|██████████| 939/939 [05:00<00:00,  3.12it/s]

...

Val loss: 0.1236 | IoU: 0.8780 | Dice: 0.9270 | Acc: 0.9780
Precision: 0.9302 | Recall: 0.9299 | F1: 0.9317


Train epoch 14: 100%|██████████| 939/939 [04:59<00:00,  3.13it/s]

Epoch 14 - train avg loss: 0.0772 | IoU: 0.9021 | Dice: 0.9438 | Acc: 0.9849


Val loss: 0.1222 | IoU: 0.8814 | Dice: 0.9293 | Acc: 0.9786
Precision: 0.9344 | Recall: 0.9325 | F1: 0.9325


Train epoch 15:   3%|▎         | 25/939 [00:08<04:43,  3.22it/s]

In [ ]:
save_path = "/content/drive/MyDrive/Data Mining/Project/Model/deeplabv3_mbv3_raw.pth"

torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, save_path)

print(f"Model saved to {save_path}")